# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import geopandas as gpd
import requests

# Import API key
from api_keys import geoapify_key

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kodiak,57.7900,-152.4072,45.91,93,100,13.80,US,1698509507
1,1,waitangi,-43.9535,-176.5597,48.81,83,35,5.99,NZ,1698509507
2,2,sawakin,19.1059,37.3321,86.25,67,22,14.12,SD,1698509508
3,3,port-aux-francais,-49.3500,70.2167,41.29,98,100,17.76,TF,1698509508
4,4,fortuna,40.5982,-124.1573,40.12,100,40,9.22,US,1698509509


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [28]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      alpha = 0.6,
                                      size = "Humidity",
                                      tiles = "OSM"
                                     )

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_data_df = city_data_df.sort_values(by='Cloudiness', axis=0, ascending=True)

# Drop any rows with null values
ideal_data_df.dropna(axis=0)

# Display sample data
ideal_data_df.head(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
273,273,saint-francois,46.4154,3.9054,62.65,60,0,8.05,FR,1698509625
128,128,cabo san lucas,22.8909,-109.9124,79.52,67,0,3.44,MX,1698509353
123,123,laguna,38.4210,-121.4238,52.97,83,0,6.91,US,1698509564
359,359,red hill,33.9996,-79.2581,80.91,67,0,5.26,US,1698509663
367,367,tura,25.5198,90.2201,66.83,73,0,3.60,IN,1698509666
368,368,sooke,48.3746,-123.7276,41.58,73,0,13.80,CA,1698509667
392,392,springbok,-29.6643,17.8865,69.31,14,0,21.30,ZA,1698509676
395,395,dawson creek,55.7666,-120.2362,26.24,63,0,10.36,CA,1698509677
96,96,lihue,21.9789,-159.3672,70.84,77,0,9.22,US,1698509412
95,95,alvand,36.3187,49.1678,57.63,32,0,5.41,IR,1698509554


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.insert(5, "Hotel Name", "")

# Display sample data
hotel_df.head(20)

,City,Country,Lat,Lng,Humidity,Hotel Name
273,saint-francois,FR,46.4154,3.9054,60,
128,cabo san lucas,MX,22.8909,-109.9124,67,
123,laguna,US,38.4210,-121.4238,83,
359,red hill,US,33.9996,-79.2581,67,
367,tura,IN,25.5198,90.2201,73,
368,sooke,CA,48.3746,-123.7276,73,
392,springbok,ZA,-29.6643,17.8865,14,
395,dawson creek,CA,55.7666,-120.2362,63,
96,lihue,US,21.9789,-159.3672,77,
95,alvand,IR,36.3187,49.1678,32,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [23]:
# Set parameters to search for a hotel
radius = 10000 
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_key
         }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    Latitude = hotel_df.loc[index, "Lat"]
    Longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Longitude},{Latitude},{radius}"
    params["bias"] = f"proximity:{Longitude},{Latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

Starting hotel search
saint-francois - nearest hotel: Chez Lily
cabo san lucas - nearest hotel: Comfort Rooms
laguna - nearest hotel: Holiday Inn Express & Suites
red hill - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
sooke - nearest hotel: Best Western Premier Prestige Oceanfront Resort
springbok - nearest hotel: Elkoweru Guesthouse
dawson creek - nearest hotel: Comfort Inn
lihue - nearest hotel: Kauai Palms
alvand - nearest hotel: هتل لاله
petropavlovsk-kamchatsky - nearest hotel: ООО Постоялый двор
al kharijah - nearest hotel: Kharga Hotel
tuscaloosa - nearest hotel: Embassy Suites Tuscaloosa
badger - nearest hotel: Hotel North Pole
bethel - nearest hotel: Hampton Inn Danbury
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
raymondville - nearest hotel: No hotel found
anyang - nearest hotel: 安阳宾馆
coos bay - nearest hotel: No hotel found
holualoa - nearest hotel: Kona Hotel
kapa'a - nearest hotel: Pono Kai Resort
dakhla - nearest hotel: Hot

hilo - nearest hotel: Dolphin Bay Hotel
hawaiian paradise park - nearest hotel: No hotel found
tralee - nearest hotel: The Ashe Hotel
bodden town - nearest hotel: Turtle Nest Inn
howrah - nearest hotel: Sun India Guest House
fortuna - nearest hotel: Comfort Inn & Suites Redwood Country
pisco - nearest hotel: La Portada
port hedland - nearest hotel: The Esplanade Hotel
vila velha - nearest hotel: Hotel Prainha
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
nuku'alofa - nearest hotel: City Hotel
bozdar wada - nearest hotel: No hotel found
olinda - nearest hotel: Fidji Motel
palmas - nearest hotel: Hotel Triângulo Mineiro
rabo de peixe - nearest hotel: Pico do Refugio
tamanrasset - nearest hotel: فندق الأمان
cozumel - nearest hotel: Hotel Isla Esmeralda
longboat key - nearest hotel: Zota Beach Resort
ribeira grande - nearest hotel: Hospedaria JSF
pitimbu - nearest hotel: Pousada Brisa Mar
nogliki - nearest hotel: Ноглики
tres arroyos - nearest hotel: Andrea Hotel
dahab - near

punta arenas - nearest hotel: Hotel Lacolet
nanning - nearest hotel: No hotel found
buala - nearest hotel: No hotel found
n'dalatando - nearest hotel: Hotel Terminus
sao vicente - nearest hotel: Pousada Vitória
vallegrande - nearest hotel: Hotel Santa Cruz
kuala belait - nearest hotel: Hotel Sentosa
ancud - nearest hotel: Cabañas
selfoss - nearest hotel: Miðgarður
aasiaat - nearest hotel: SØMA
ciudad lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
kalmunai - nearest hotel: Saji new food
swift current - nearest hotel: Days Inn
san jose del guaviare - nearest hotel: El Portico
vilhena - nearest hotel: No hotel found
caleta de carquin - nearest hotel: Hotel Mi Choacan
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
agapovka - nearest hotel: No hotel found
mendi - nearest hotel: Kiburu Lodge
camacupa - nearest hotel: No hotel found
neya - nearest hotel: No hotel found
pathein - nearest hotel: Taan Taan Ta
minas de marcona - nearest hotel: No hotel found
san fernando de apur

In [24]:
# Display sample data
hotel_df.head(20)

,City,Country,Lat,Lng,Humidity,Hotel Name
273,saint-francois,FR,46.4154,3.9054,60,Chez Lily
128,cabo san lucas,MX,22.8909,-109.9124,67,Comfort Rooms
123,laguna,US,38.4210,-121.4238,83,Holiday Inn Express & Suites
359,red hill,US,33.9996,-79.2581,67,No hotel found
367,tura,IN,25.5198,90.2201,73,No hotel found
368,sooke,CA,48.3746,-123.7276,73,Best Western Premier Prestige Oceanfront Resort
392,springbok,ZA,-29.6643,17.8865,14,Elkoweru Guesthouse
395,dawson creek,CA,55.7666,-120.2362,63,Comfort Inn
96,lihue,US,21.9789,-159.3672,77,Kauai Palms
95,alvand,IR,36.3187,49.1678,32,هتل لاله


In [25]:
name_address

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'name': 'Shelikof Lodge',
    'country': 'United States',
    'country_code': 'us',
    'state': 'Alaska',
    'county': 'Kodiak Island',
    'city': 'Kodiak',
    'postcode': '99615',
    'street': 'East Hillcrest Avenue',
    'lon': -152.4070946,
    'lat': 57.7913327,
    'state_code': 'AK',
    'formatted': 'Shelikof Lodge, East Hillcrest Avenue, Kodiak, AK 99615, United States of America',
    'address_line1': 'Shelikof Lodge',
    'address_line2': 'East Hillcrest Avenue, Kodiak, AK 99615, United States of America',
    'categories': ['accommodation', 'accommodation.hotel'],
    'details': [],
    'datasource': {'sourcename': 'openstreetmap',
     'attribution': '© OpenStreetMap contributors',
     'license': 'Open Database Licence',
     'url': 'https://www.openstreetmap.org/copyright',
     'raw': {'name': 'Shelikof Lodge',
      'osm_id': 2196693223,
      'tourism': 'hotel',
      'osm_type': 'n'

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [26]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)